In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mayo-clinic-strip-ai/sample_submission.csv
/kaggle/input/mayo-clinic-strip-ai/train.csv
/kaggle/input/mayo-clinic-strip-ai/test.csv
/kaggle/input/mayo-clinic-strip-ai/other.csv
/kaggle/input/mayo-clinic-strip-ai/other/314263_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/0b33d7_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/54334d_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/84c279_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/cca9eb_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/880188_1.tif
/kaggle/input/mayo-clinic-strip-ai/other/2e3078_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/9daa03_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/065053_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/8acdfa_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/91530f_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/a2253f_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/f5eb8f_1.tif
/kaggle/input/mayo-clinic-strip-ai/other/ab830f_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/d17ad3_2.tif

In [5]:
import os
import glob
import pandas as pd
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from PIL import Image
import matplotlib.pyplot as plt
# 设置数据集路径
DATASET_FOLDER = "/kaggle/input/mayo-clinic-strip-ai/"
DATASET_SMALL_FOLDER = "/kaggle/input/newtrain"

# 验证路径是否存在
if not os.path.exists(DATASET_FOLDER):
    print(f"警告: {DATASET_FOLDER} 不存在")

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import keras.backend as K #to define custom loss function

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from pprint import pprint
from collections import defaultdict
import openslide
from openslide import OpenSlide

from glob import glob

from sklearn.model_selection import train_test_split
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.layers import GlobalMaxPooling2D
from keras.models import load_model

print(keras.__version__)


3.3.3


In [7]:
import random

train_df = pd.read_csv('../input/mayo-clinic-strip-ai/train.csv')
test_df  = pd.read_csv('../input/mayo-clinic-strip-ai/test.csv')

# Specify patient_ids to remove
patient_ids_to_remove = ['006388', '008e5c', '00c058', '01adc5']

# Filter out the rows with specified patient_ids
train_df = train_df[~train_df['patient_id'].isin(patient_ids_to_remove)].reset_index(drop=True)
train_df = train_df.drop_duplicates(subset=['patient_id'])

df1 = train_df[train_df['label'] == 'CE']
df2 = train_df[train_df['label'] == 'LAA']
#adjust n to change number of CE data
sampled= df1.sample(n=200, random_state=42)
train_df = pd.concat([sampled, df2],ignore_index=True)
# Print the cleaned DataFrame
print("Cleaned DataFrame:")
print(train_df.head())
train_df['label'].value_counts()

Cleaned DataFrame:
   image_id  center_id patient_id  image_num label
0  a9573c_0          8     a9573c          0    CE
1  140d1a_0          6     140d1a          0    CE
2  dfd99d_0          7     dfd99d          0    CE
3  e8dceb_0         11     e8dceb          0    CE
4  512856_0         11     512856          0    CE


label
CE     200
LAA    173
Name: count, dtype: int64

In [8]:
train_df['patient_id'].nunique

<bound method IndexOpsMixin.nunique of 0      a9573c
1      140d1a
2      dfd99d
3      e8dceb
4      512856
        ...  
368    fd368e
369    fd684f
370    fde9ba
371    fe9bec
372    ffec5c
Name: patient_id, Length: 373, dtype: object>

In [9]:
train_images = glob("/kaggle/input/mayo-clinic-strip-ai/train/*")
test_images = glob("/kaggle/input/mayo-clinic-strip-ai/test/*")
other_images = glob("/kaggle/input/mayo-clinic-strip-ai/other/*")
print(f"Number of images in a training set: {len(train_images)}")
print(f"Number of images in a training set: {len(test_images)}")
print(f"Number of other: {len(other_images)}")

# Filtering out images based on the cleaned patient_ids in train_df
images_to_remove = [
    '/kaggle/input/mayo-clinic-strip-ai/train/006388_0.tif',
    '/kaggle/input/mayo-clinic-strip-ai/train/008e5c_0.tif',
    '/kaggle/input/mayo-clinic-strip-ai/train/00c058_0.tif',
    '/kaggle/input/mayo-clinic-strip-ai/train/01adc5_0.tif',
]

# Remove images associated with the patient_ids
train_images = [img for img in train_images if img not in images_to_remove]

# Check the total number of images after deletion
total_images_after_deletion = len(train_images)
print("Total number of images after deletion:", total_images_after_deletion)

# Print the paths of the cleaned list of images
print("First 5 image paths after deletion:")
print(train_images[:5])

Number of images in a training set: 754
Number of images in a training set: 4
Number of other: 396
Total number of images after deletion: 750
First 5 image paths after deletion:
['/kaggle/input/mayo-clinic-strip-ai/train/a4c7df_0.tif', '/kaggle/input/mayo-clinic-strip-ai/train/f9fc6b_0.tif', '/kaggle/input/mayo-clinic-strip-ai/train/0d718a_0.tif', '/kaggle/input/mayo-clinic-strip-ai/train/86e319_1.tif', '/kaggle/input/mayo-clinic-strip-ai/train/54838a_0.tif']


In [10]:
train_df["file_path"] = train_df["image_id"].apply(lambda x: "../input/mayo-clinic-strip-ai/train/" + x + ".tif")
test_df["file_path"]  = test_df["image_id"].apply(lambda x: "../input/mayo-clinic-strip-ai/test/" + x + ".tif")

In [11]:
# labelling CE class as 1 and LAA as 0
train_df["target"] = train_df["label"].apply(lambda x : 1 if x=="CE" else 0)
train_df.head()

,image_id,center_id,patient_id,image_num,label,file_path,target
0,a9573c_0,8,a9573c,0,CE,../input/mayo-clinic-strip-ai/train/a9573c_0.tif,1
1,140d1a_0,6,140d1a,0,CE,../input/mayo-clinic-strip-ai/train/140d1a_0.tif,1
2,dfd99d_0,7,dfd99d,0,CE,../input/mayo-clinic-strip-ai/train/dfd99d_0.tif,1
3,e8dceb_0,11,e8dceb,0,CE,../input/mayo-clinic-strip-ai/train/e8dceb_0.tif,1
4,512856_0,11,512856,0,CE,../input/mayo-clinic-strip-ai/train/512856_0.tif,1


In [12]:
%%time
train_df["file_path"] = train_df["image_id"].apply(lambda x: "../input/mayo-clinic-strip-ai/train/" + x + ".tif")

def preprocess(image_path):
    slide=OpenSlide(image_path)
    region= (2500,2500)    
    size  = (5000, 5000)
    image = slide.read_region(region, 0, size)
    image = image.resize((128, 128))
    image = np.array(image)    
    return image

X_train=[]
for i in tqdm(train_df['file_path']):
    x1=preprocess(i)
    X_train.append(x1)

Y_train=[]    
Y_train=train_df['target']


100%|██████████| 373/373 [28:37<00:00,  4.60s/it]

CPU times: user 7min 53s, sys: 1min 57s, total: 9min 51s
Wall time: 28min 37s


In [13]:
X_train=np.array(X_train)
X_train=X_train/255.0
Y_train = np.array(Y_train)

In [14]:
!pip install tensorflow numpy pandas tqdm

In [15]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB6
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [16]:
print("Y_train shape:", Y_train.shape)

Y_train shape: (373,)


In [17]:
print("Unique values in Y_train:", np.unique(Y_train))

Unique values in Y_train: [0 1]


In [18]:
from tensorflow.keras.utils import to_categorical

# 将整数标签转换为 one-hot 编码
Y_train = to_categorical(Y_train, num_classes=2)

print("Y_train shape after one-hot encoding:", Y_train.shape)
# 输出: (373, 2)

Y_train shape after one-hot encoding: (373, 2)


In [19]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X_train,Y_train, test_size=0.3, random_state=42)
x_train,x_val,y_train,y_val=train_test_split(X_train,Y_train, test_size=0.5, random_state=42)

In [20]:
from tensorflow.keras.utils import to_categorical



# 数据增强
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# 生成增强数据
train_generator = datagen.flow(
    X_train, 
    Y_train, 
    batch_size=32  # 去掉 class_mode
)

In [21]:
from tensorflow.keras.applications import EfficientNetB6
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical



# 加载 EfficientNet-B6 的预训练模型
base_model = EfficientNetB6(weights=None, include_top=False, input_shape=(128, 128, 4))

# 冻结预训练模型的卷积层
base_model.trainable = False

# 添加自定义分类头
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(Y_train.shape[1], activation='softmax')(x)  # 输出层

# 定义完整模型
model = Model(inputs=base_model.input, outputs=predictions)

# 编译模型
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# 打印模型结构
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 128, 128,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 128, 128,  │          9 │ rescaling[0][0]   │
│ (Normalization)     │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 129, 129,  │          0 │ normalization[0]… │
│ (ZeroPadding2D)     │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 64, 64,    │      2,016 │ stem_conv_pad[0]… │
│                     │ 56)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 64, 64,    │        224 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 56)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 64, 64,    │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 56)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 64, 64,    │        504 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 56)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 64, 64,    │        224 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 56)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 64, 64,    │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 56)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 56)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 56)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 14)  │        798 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 56)  │        840 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 64, 64,    │          0 │ block1a_activati… │
│ (Multiply)          │ 56)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 64, 64,    │      1,792 │ block1a_se_excit… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 64, 64,    │        128 │ block1a_project_

 Total params: 41,551,243 (158.51 MB)

 Trainable params: 590,594 (2.25 MB)

 Non-trainable params: 40,960,649 (156.25 MB)

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 定义回调函数
early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)  # 监控训练集的 loss
model_checkpoint = ModelCheckpoint('efficientnet_b6_best_model.keras', save_best_only=True, monitor='loss')  # 保存基于训练集 loss 的最优模型

# 开始训练
history = model.fit(
    train_generator,  # 训练数据生成器
    epochs=20,        # 训练轮数
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/20


I0000 00:00:1733237051.077340     111 service.cc:145] XLA service 0x7ad854003ab0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733237051.077417     111 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


 1/12 ━━━━━━━━━━━━━━━━━━━━ 18:18 100s/step - accuracy: 0.5312 - loss: 0.6931

I0000 00:00:1733237125.067487     111 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


12/12 ━━━━━━━━━━━━━━━━━━━━ 172s 7s/step - accuracy: 0.4669 - loss: 0.6933
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 260ms/step - accuracy: 0.5526 - loss: 0.6921
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 254ms/step - accuracy: 0.5389 - loss: 0.6909
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 247ms/step - accuracy: 0.5127 - loss: 0.6927
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 258ms/step - accuracy: 0.5334 - loss: 0.6910
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.5058 - loss: 0.6946
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.5641 - loss: 0.6885
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 248ms/step - accuracy: 0.5530 - loss: 0.6886
Epoch 9/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.5405 - loss: 0.6897
Epoch 10/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.4632 - loss: 0.7007
Epoch 11/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 299ms/step - accuracy: 0.5030 - loss: 0.6940
Epoch 12/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 80ms/step - accuracy: 0.54